In [14]:
import pandas as pd
import pickle
import re
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.base import BaseEstimator, TransformerMixin

# Load dataset
df = pd.read_csv("data/updatedResumeDataSet.csv")

# Custom text cleaning and preprocessing class
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        def clean_text(text):
            text = text.lower()
            text = re.sub(r'http\S+|www\S+|https\S+', '', text)
            text = re.sub(r'\S+@\S+', '', text)
            text = re.sub(r'\b\d{1,3}[-./]?\d{1,3}[-./]?\d{1,4}\b', '', text)
            text = re.sub(r'\[.*?\]', ' ', text)
            text = re.sub("\\W", " ", text)
            text = re.sub('<.*?>+', ' ', text)
            text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
            text = re.sub('\n', ' ', text)
            text = re.sub(r'\w*\d\w*', ' ', text)
            text = re.sub(r'[^a-zA-Z]', ' ', text)
            text = re.sub(r'\s+', ' ', text)
            tokens = text.split()
            tokens = [word for word in tokens if word not in ENGLISH_STOP_WORDS]
            return ' '.join(tokens).strip()

        return [clean_text(text) for text in X]

# Apply preprocessing
preprocessor = TextPreprocessor()
df["Resume"] = preprocessor.fit_transform(df["Resume"])

# Encode labels
le = LabelEncoder()
le.fit(df["Category"])
df["Category"] = le.transform(df["Category"])

# Save LabelEncoder
pickle.dump(le, open('models/label_encoder.pkl', 'wb'))

# TF-IDF Vectorization
tfidf = TfidfVectorizer()
tfidf.fit(df['Resume'])
transformed_text = tfidf.transform(df['Resume'])

# Split data
X_train, X_test, y_train, y_test = train_test_split(transformed_text, df['Category'], test_size=0.2, random_state=42)

# Train classifier
clf = OneVsRestClassifier(KNeighborsClassifier())
clf.fit(X_train, y_train)

# Predict and evaluate
ypred = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, ypred))

# Save models
pickle.dump(tfidf, open('models/tfidf.pkl', 'wb'))
pickle.dump(clf, open('models/clf.pkl', 'wb'))

# Predict category for a new resume
def predict_category(resume_text):
    cleaned_resume = preprocessor.transform([resume_text])[0]
    input_features = tfidf.transform([cleaned_resume])
    prediction_id = clf.predict(input_features)[0]
    category_name = le.inverse_transform([prediction_id])[0]
    return category_name

Accuracy: 0.9844559585492227
